In [21]:
import requests
import json
import os
import pandas as pd

#### Which traditional stat is most important?
* Data Needed: Player name, points, assists, rebounds, FG%, team margin

* Find correlation between winning and an imputed stats (points, assists, rebounds, FG%).
  Scatter Plot

In [22]:
filepath = os.path.join('nba_season_stats_kaggle.csv')

tradStats_df = pd.read_csv(filepath)
tradStats_df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [23]:
tradStats_df.columns

Index(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2',
       'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [24]:
tradStats_df['Year'].value_counts()

2015.0    651
2011.0    625
2014.0    611
2008.0    595
2017.0    595
         ... 
1959.0    110
1955.0    109
1957.0    107
1956.0    106
1961.0    104
Name: Year, Length: 68, dtype: int64

In [29]:
Reduced_df = tradStats_df.loc[:, ['Player', 'Year','PTS', 'AST', 'ORB', 'DRB', 'TRB', 'FG', 'FG%']]
Reduced_df

,Player,Year,PTS,AST,ORB,DRB,TRB,FG,FG%
0,Curly Armstrong,1950.0,458.0,176.0,NaN,NaN,NaN,144.0,0.279
1,Cliff Barker,1950.0,279.0,109.0,NaN,NaN,NaN,102.0,0.372
2,Leo Barnhorst,1950.0,438.0,140.0,NaN,NaN,NaN,174.0,0.349
3,Ed Bartels,1950.0,63.0,20.0,NaN,NaN,NaN,22.0,0.256
4,Ed Bartels,1950.0,59.0,20.0,NaN,NaN,NaN,21.0,0.256
...,...,...,...,...,...,...,...,...,...
24686,Cody Zeller,2017.0,639.0,99.0,135.0,270.0,405.0,253.0,0.571
24687,Tyler Zeller,2017.0,178.0,42.0,43.0,81.0,124.0,78.0,0.494
24688,Stephen Zimmerman,2017.0,23.0,4.0,11.0,24.0,35.0,10.0,0.323
24689,Paul Zipser,2017.0,240.0,36.0,15.0,110.0,125.0,88.0,0.398


In [31]:
# only pick 10 years from 2007 to 2017

Reduced_df.loc[Reduced_df[['Year'] >= 2007,:]]


TypeError: '>=' not supported between instances of 'list' and 'str'